In [2]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from pydantic import BaseModel, Field
import dotenv
import os

dotenv.load_dotenv()

True

In [26]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

file_path = os.listdir("files/")[1]

In [27]:
# 리트리버 생성

def embedding_file(file_path):
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=300,
        chunk_overlap=50,
        separators=["\n\n"],
    )

    loader = PyPDFLoader(f"./files/{file_path}")
    docs = loader.load_and_split(text_splitter=splitter)

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(docs, embeddings)
    retriever = vector_store.as_retriever()

    return retriever

In [28]:
retriever = embedding_file(file_path)

In [ ]:
# retriever : Annotated[VectorStoreRetriever, "VectorStore Retriever"]

In [299]:
class State(TypedDict):
    query : Annotated[str, "User Question"]
    answer : Annotated[str, "LLM Response"]
    document : Annotated[Document, "Retrieve Response"]
    init_state : Annotated[bool, "Init Checker"]

In [300]:
# Start -> get_initial_state

def get_initial_state(state:State):
    if state.get("init_state", False):
        print("====== Already State ======")
        return 
    else:
        print("====== State Init Complete ======")
        return {"answer":"",
                "init_state":True,
                }

In [301]:
def create_document(state:State):
    docs = retriever.invoke(state["query"])

    docs = "\n\n".join(doc.page_content for doc in docs)

    return {"document":docs}

In [302]:
def create_prompt(state:State):
    prompt = ChatPromptTemplate.from_messages([
                ("system", 
                """
                context : {context}

                당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
                ---
                ### 규칙 ###
                1. 주어진 context만을 이용하여 답변해야합니다. 
                2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
                3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
                4. 항상 친절한 말투로 응대합니다.
                5. 하이퍼 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요. 아래 예시를 참고하여 서식을 맞추세요.
                **하이퍼 링크 예시**
                5-1. [스타벅스 구역삼사거리점](https://naver.me/FV7K6xTM) 입니다.
                5-2. [화목순대국](https://naver.me/FQVGK6TZ) 입니다.
                5-3. [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5) 입니다.
                ---
                """),
                ("human", "{query}")
            ])
    
    chain = prompt | llm

    return {"answer": chain.invoke({"context" : state["document"],
                                    "query": state["query"]})}

In [303]:
graph_builder = StateGraph(State)

In [304]:
graph_builder.add_node("init", get_initial_state);
graph_builder.add_node("format_docs", create_document);
graph_builder.add_node("prompt1", create_prompt);

graph_builder.add_edge(START, "init");
graph_builder.add_edge("init", "format_docs");
graph_builder.add_edge("format_docs", "prompt1");
graph_builder.add_edge("prompt1", END); 

In [305]:
graph = graph_builder.compile()

In [ ]:
graph

In [289]:
invoke_result = graph.invoke({"query":"개강하는 날짜는 언제인가요?"})

====== State Init Complete ======


In [295]:
invoke_result

{'query': '개강하는 날짜는 언제인가요?',
 'answer': AIMessage(content='아이펠 데이터사이언티스트 3기 개강일은 2024년 9월 25일입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 3921, 'total_tokens': 3950, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ccb0b20-a39c-4535-9802-0dc6c8681cfa-0', usage_metadata={'input_tokens': 3921, 'output_tokens': 29, 'total_tokens': 3950, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'document': '아이펠  데이터사이언티스트  3 기\n2A4 . 내일배움카드  실수령  후  다음  날  훈련생  등록이  가능하여  개강일  전일 ( 평일  기준 ) 까지  발급이  가능해야  합니다 . 가능한  아이펠  서류제출과  함께  바로  내일배움카드  신청을  해\n주시는  게  가장  안전합니다 ! 부득이  하게 

In [296]:
invoke_result = graph.invoke({"query":"오프라인강의인가요?"})

====== State Init Complete ======


In [297]:
invoke_result

{'query': '오프라인강의인가요?',
 'answer': AIMessage(content='아이펠 교육과정은 100% 실시간 온라인 과정으로 화상 플랫폼(ZEP)에서 진행됩니다. 하지만 희망하시는 경우, 오프라인 학습 공간(모두의연구소 강남 / 역삼캠퍼스)을 무료로 제공해드리고 있습니다. 공간 사용방법은 입학 후 별도로 안내드릴 예정입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 3920, 'total_tokens': 3999, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 3840}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-18819a83-2cda-454f-9571-85cf6d4504f8-0', usage_metadata={'input_tokens': 3920, 'output_tokens': 79, 'total_tokens': 3999, 'input_token_details': {'audio': 0, 'cache_read': 3840}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'document': '아이펠  데이터사이언티스트  3 기\n2A4 . 내일배움카드  실수령  후  다음  날  훈련생  등록이  가능하여  개강일  전